In [8]:
import os, sys
sys.path.append(os.path.join("../.."))

import privugger.transformer.datastructures as pvds
from privugger.transformer.discrete import *
from privugger.transformer.continuous import *
from privugger.transformer.program_output import *

from privugger.transformer.method import *
from privugger.measures.mutual_information import *

import arviz as az
import pymc3 as pm

import fastprogress
master_bar,progress_bar = fastprogress.force_console_behavior()

AttributeError: module 'fastprogress' has no attribute 'force_console_behavior'

In [2]:
#####################
## PATH TO PROGRAM ##
#####################
program = "program_to_analyze.py"

In [5]:
#################
## AGE EXAMPLE ##
#################

# Database size
N    = 10

# Specify distributions
age  = Normal(mu=55.2, std=3.5, num_elements=N)
name = DiscreteUniform(0, 5, num_elements=N)
    
age_alice  = Uniform(lower=0, upper=100)
name_alice = DiscreteUniform(0, 5) 


# Create dataset and specify program output
# For now output type can be: Int, Float, List(Float), List(Int)
ds = pvds.Dataset(input_specs = [age, age_alice, name, name_alice],
                  var_names   = ["age", "alice_age", "name", "name_alice"],
                  program_output = Float)

# Call infer
trace = infer(ds,  program, draws=1000, cores=4)

import theano
import theano.tensor as tt
import numpy as np


def method(age, age_alice, name, name_alice):

    @theano.compile.ops.as_op(itypes=[tt.dvector, tt.dscalar, tt.lvector,
        tt.lscalar], otypes=[tt.dscalar])
    def alpha(age, age_alice, name, name_alice):
        return np.array((sum(age) + age_alice) / (age.size + 1))
    return alpha(age, age_alice, name, name_alice)



[age ~ Normal, alice_age ~ Uniform, name ~ DiscreteUniform, name_alice ~ DiscreteUniform]


../../privugger/transformer/method.py:149: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  trace = pm.sample(draws=draws, chains=chains, cores=cores)


Multiprocess sampling (2 chains in 4 jobs)


CompoundStep


...

>>Metropolis: [name_alice]


>>Metropolis: [name]


Sampling 2 chains for 1_000 tune and 1_000 draw iterations (2_000 + 2_000 draws total) took 2 seconds.


The rhat statistic is larger than 1.05 for some parameters. This indicates slight problems during sampling.


The estimated number of effective samples is smaller than 200 for some parameters.


In [5]:
age = trace['alice_age']
out = trace['output']

mi_sklearn(age, out, n_neigh=40)

array([1.48835712])

In [4]:

N = 10

#alice_name      = Constant(0)
#alice_age       = Constant(25)
#alice_zip       = Constant(50)
#alice_gender    = Constant(1) 
#alice_diagnosis = Constant(1) 
    
    
names = DiscreteUniform(0, 5, num_elements=N)
ages  = Normal(mu=55.2, std=3.5, num_elements=N)

zips = DiscreteUniform(0, 100, num_elements=N)

genders = Bernoulli(p=.5, num_elements=N)   
diagnoses = Bernoulli(p=.2,num_elements=N)

ds = pvds.Dataset(input_specs = [names, ages, zips, genders, diagnoses], 
                  var_names=["names", "ages", "zips", "genders", "diagnoses"])


trace = infer(ds, stack=True)

[names, ages, zips, genders, diagnoses]
names
ages
zips
genders
diagnoses


Multiprocess sampling (2 chains in 2 jobs)
CompoundStep
>CompoundStep
>>Metropolis: [zips]
>>Metropolis: [names]
>NUTS: [ages]
>BinaryGibbsMetropolis: [genders, diagnoses]


Sampling 2 chains for 1_000 tune and 500 draw iterations (2_000 + 1_000 draws total) took 3 seconds.
The rhat statistic is larger than 1.2 for some parameters.
The estimated number of effective samples is smaller than 200 for some parameters.
